# Project: Pressure sensor placement for leakage detection

### Introduction

(This notebook was originally based on the epyt-flow tutorial for leakage detection, which can be found here: [Tutorials](https://github.com/WaterFutures/EPyT-and-EPyT-Flow-Tutorial))

**Authors:** Timo Beckmann, Tobias Kroecker

This notebook documents a research project focused on evaluating two approaches to different critical aspects of leakage detection in water distribution networks:

1. **Sensor Placement Strategy**: We implement and evaluate a sensor placement algorithm based on information theory. This approach aims to optimize the placement of pressure sensors by maximizing relevance and minimizing redundancy, ensuring effective network coverage. The evaluation includes varying the number of sensors to analyze the impact of sensor density on detection performance.

2. **Leakage Detection Method**: A machine learning-based leakage detection approach is implemented using a bagging classifier with decision trees. This method leverages ensemble learning to enhance robustness and accuracy in detecting anomalies such as leakages in the network.

3. **Datasets**: 
    - **LeakDB**: A benchmark dataset providing realistic scenarios for leakage detection in water distribution networks, used to evaluate both the sensor placement strategy and the leakage detection method.
    - **BattLeDIM**: A dataset offering realistic demand patterns and scenarios, used to simulate and validate the sensor placement strategy.

4. **Evaluation Goals**: The primary objective is to assess the performance of the leakage detection method and the effectiveness of the sensor placement strategy. This includes:
    - Analyzing the detection accuracy of the machine learning model.
    - Investigating the influence of different sensor placement sizes on detection performance.

By combining theoretical insights from information theory and machine learning with practical applications, this notebook provides a comprehensive evaluation of the interplay between sensor placement and leakage detection in water distribution networks.

## Imports and method definitions

### Imports

In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=ImportWarning)
warnings.filterwarnings("ignore", category=UserWarning)

import numpy as np
import matplotlib.pyplot as plt
import sklearn
import epyt_flow
import pandas as pd
import random
import json
import datetime
import os

from sklearn.metrics import confusion_matrix
from sklearn.metrics import mutual_info_score
from sklearn.metrics import classification_report
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import train_test_split
from epyt_flow.data.benchmarks import leakdb
from epyt_flow.data.benchmarks import load_leakdb_scenarios
from epyt_flow.data.benchmarks.leakdb_data import HANOI_LEAKAGES
from epyt_flow.data.benchmarks.battledim import load_scenario, load_scada_data
from epyt_flow.simulation import ScenarioSimulator
from epyt_flow.utils import to_seconds, time_points_to_one_hot_encoding
from joblib import Parallel, delayed, parallel_backend

### Implementation of the sensor placement algorithm

In [2]:
# Assume that we know which specific nodes we want to watch:
def calc_Relevance (node_pressures, relevant_index):
    relevances = {}
    for i in range(len(node_pressures)):
        relevances[i] = mutual_info_score(node_pressures[relevant_index], node_pressures[i])
    return relevances

# calcuate relevance over all nodes:

def calc_Relevances (node_pressures):
    relevances = {}
    for i in range(len(node_pressures)):
        for j in range(i + 1, len(node_pressures)):
            m_i_score = mutual_info_score(node_pressures[i], node_pressures[j])
            if i not in relevances:
                relevances[i] = m_i_score
            else:
                relevances[i] += m_i_score
            if j not in relevances:
                relevances[j] = m_i_score
            else:
                relevances[j] += m_i_score
    return relevances
    
def calc_Redundance (x_pressures, sensor_pressures):
    redundancy = 0
    for pressure in sensor_pressures:
        redundancy += mutual_info_score(x_pressures, pressure)
    redundancy /= len(sensor_pressures)
    return redundancy

In [3]:
def calc_Relevances_parallel(node_pressures, n_jobs=1):
    relevances = Parallel(n_jobs=n_jobs)(delayed(calc_Relevance)(node_pressures, i) for i in range(len(node_pressures)))
    return {i: sum(relevance[i] for relevance in relevances) for i in range(len(node_pressures))}

In [4]:
def calc_sensor_placement(node_pressures, num_sensors = 15): # do we know which nodes we want to monitor? junctions? all of them?
    remaining_nodes = list(range(len(node_pressures)))
    relevances = calc_Relevances(node_pressures)  # Assume we want to know the relevances in regard to all nodes
    Sensor_placement = [max(relevances, key=relevances.get)]
    remaining_nodes.remove(Sensor_placement[-1])
    sensor_pressures = [node_pressures[Sensor_placement[-1]]]
    no_redundance_nodes = [a for a in remaining_nodes if calc_Redundance(node_pressures[a], node_pressures[Sensor_placement]) == 0]
    while no_redundance_nodes: # add nodes of highest relevance without any redundance
        if (len(Sensor_placement) >= num_sensors):
            break
        tmp_relevances = relevances.copy()
        for i in range(len(tmp_relevances.keys())): # remove nodes that have been added to the sensors and nodes with redundance
            if i in Sensor_placement:
                del tmp_relevances[i]
                continue
            if i not in no_redundance_nodes:
                del tmp_relevances[i]
        Sensor_placement.append(max(tmp_relevances, key=relevances.get))
        remaining_nodes.remove(Sensor_placement[-1])
        sensor_pressures.append(node_pressures[Sensor_placement[-1]])
        no_redundance_nodes = [a for a in remaining_nodes if calc_Redundance(node_pressures[a], sensor_pressures) == 0]
    remaining_relevances = list(map(relevances.get, remaining_nodes))
    while not(all(list(map(lambda x: x == 0, remaining_relevances)))) and remaining_nodes:
        if (len(Sensor_placement) >= num_sensors):
            break
        RRI = {}
        for i in remaining_nodes:
            RRI[i] = relevances[i]/ calc_Redundance(node_pressures[i], sensor_pressures)
        Sensor_placement.append(max(RRI, key=RRI.get))
        sensor_pressures.append(node_pressures[Sensor_placement[-1]])
        remaining_nodes.remove(Sensor_placement[-1])
        remaining_relevances = list(map(relevances.get, remaining_nodes))
    return Sensor_placement

In [5]:
def calc_sensor_placement_parallel(node_pressures, num_sensors = 15, n_jobs = 1): # do we know which nodes we want to monitor? junctions? all of them?
    remaining_nodes = list(range(len(node_pressures)))
    if n_jobs > 1:
        relevances = calc_Relevances_parallel(node_pressures, n_jobs=n_jobs)
    else:
        relevances = calc_Relevances(node_pressures)  # Assume we want to know the relevances in regard to all nodes
    Sensor_placement = [max(relevances, key=relevances.get)]
    remaining_nodes.remove(Sensor_placement[-1])
    sensor_pressures = [node_pressures[Sensor_placement[-1]]]
    Redundances = Parallel(n_jobs=n_jobs)(delayed(calc_Redundance)(node_pressures[a], node_pressures[Sensor_placement]) for a in remaining_nodes)
    no_redundance_nodes = [remaining_nodes[a] for a in range(len(remaining_nodes)) if Redundances[a] == 0]
    while no_redundance_nodes: # add nodes of highest relevance without any redundance
        if (len(Sensor_placement) >= num_sensors):
            break
        tmp_relevances = relevances.copy()
        for i in range(len(tmp_relevances.keys())): # remove nodes that have been added to the sensors and nodes with redundance
            if i in Sensor_placement:
                del tmp_relevances[i]
                continue
            if i not in no_redundance_nodes:
                del tmp_relevances[i]
        Sensor_placement.append(max(tmp_relevances, key=relevances.get))
        remaining_nodes.remove(Sensor_placement[-1])
        sensor_pressures.append(node_pressures[Sensor_placement[-1]])
        Redundances = Parallel(n_jobs=n_jobs)(delayed(calc_Redundance)(node_pressures[a], node_pressures[Sensor_placement]) for a in remaining_nodes)
        no_redundance_nodes = [remaining_nodes[a] for a in range(len(remaining_nodes)) if Redundances[a] == 0]
    remaining_relevances = list(map(relevances.get, remaining_nodes))
    while not(all(list(map(lambda x: x == 0, remaining_relevances)))) and remaining_nodes:
        if (len(Sensor_placement) >= num_sensors):
            break
        RRI = {}
        for i in remaining_nodes:
            RRI[i] = relevances[i]/ calc_Redundance(node_pressures[i], sensor_pressures)
        Sensor_placement.append(max(RRI, key=RRI.get))
        sensor_pressures.append(node_pressures[Sensor_placement[-1]])
        remaining_nodes.remove(Sensor_placement[-1])
        remaining_relevances = list(map(relevances.get, remaining_nodes))
    return Sensor_placement

### Implementation of the bagging classifier

In [6]:
# DecisionTreeClassifier with entropy criterion is in the ID3 style. Classifier architecture is based on the RDTBC-SFLO paper (https://www.researchgate.net/publication/333681237_Random_Bagging_Classifier_and_Shuffled_Frog_Leaping_Based_Optimal_Sensor_Placement_for_Leakage_Detection_in_WDS)
def mixed_model_classification_fit(n_estimators, X_train, y_train, n_jobs=1):
    with parallel_backend('threading', n_jobs=n_jobs):
        base = DecisionTreeClassifier(
            criterion='entropy',
            class_weight='balanced', 
            max_depth=5,
            max_features='sqrt',
            min_samples_leaf=5,
            min_samples_split=10,
            random_state=42
            )
        bagged = BaggingClassifier( #for faster runtime adjust parameters, especially n_estimators. n_jobs greatly improves runtime if set to a higher value
            estimator=base, 
            n_estimators=n_estimators, 
            max_samples=0.8,
            max_features=0.8,
            bootstrap=True, 
            oob_score=True
            ) 
        bagged.fit(X_train, y_train)
    return bagged

def mixed_model_classification_predict(classifier, X_test, y_test, n_jobs=1, print_report=False):
    with parallel_backend('threading', n_jobs=n_jobs):
        y_pred = classifier.predict(X_test)
    if print_report:
        print(classification_report(y_test, y_pred, labels=np.unique(y_test),
                                    target_names=[f"class_{int(c)}" for c in np.unique(y_test)], output_dict=True))
    return y_pred


### Metrics

In [7]:
# like this it can only be used for leakdb sceanrios
def early_detection_score(X_test, y_test, y_pred, scenarios):
    leaks_info = json.loads(HANOI_LEAKAGES)
    ed_score = 0
    normalizing_coef = []
    window_tolerance = 10
    detection_threshold = 0.75
    for i, scen in scenarios:
        pred = y_pred[scen]
        leaks_time_window = leakdb.__get_leak_time_windows(scen, leaks_info)

        scores = []
        for t_0, _ in leaks_time_window:
            normalizing_coef.append(1.)

            y_pred_window = pred[t_0:t_0 + window_tolerance]
            if 1 in y_pred_window and np.sum(y_pred_window) / len(y_pred_window) > detection_threshold:
                t_idx = np.argwhere(y_pred_window)[0] + 1
                scores.append(2. / (1 + np.exp((5. / window_tolerance) * t_idx)))
            else:
                scores.append(0.)
        ed_score += np.sum(scores)
    return ed_score/np.sum(normalizing_coef) if normalizing_coef else 1


In [8]:
def find_window_idx(y_test):
    diff = np.diff(y_test)
    
    # Find the indices where the difference is 1 (indicating a transition from 0 to 1)
    first_one_indices = np.where(diff == 1)[0] + 1
    
    return first_one_indices

# early detection score for that can be used with any dataset, not only leakdb
# designed for single scenarios
def general_early_detection_score(X_test, y_test, y_pred):
    leaks_idx0 = find_window_idx(y_test)

    ed_score = 0
    normalizing_coef = []
    window_tolerance = 10
    detection_threshold = 0.75
    for t_0 in leaks_idx0:
        normalizing_coef.append(1.)

        y_pred_window = y_pred[t_0:t_0 + window_tolerance]
        if 1 in y_pred_window and np.sum(y_pred_window) / len(y_pred_window) > detection_threshold:
            t_idx = np.argwhere(y_pred_window)[0] + 1
            ed_score += 2. / (1 + np.exp((5. / window_tolerance) * t_idx))
        else:
            ed_score += 0.
    return ed_score / np.sum(normalizing_coef) if normalizing_coef else 1

In [9]:
def evaluate_and_collect(clf, X_test, y_test, verbose=False):
    if verbose:
        y_pred = mixed_model_classification_predict(clf, X_test, y_test, print_report=True)
    else:
        y_pred = mixed_model_classification_predict(clf, X_test, y_test, print_report=False)
    rpt = classification_report(y_test, y_pred, output_dict=True, zero_division=0)
    probabilities = clf.predict_proba(X_test)
    uniq = np.unique(y_test)
    if len(uniq) == 2:
        ll  = log_loss(y_test, probabilities)
        rc = roc_auc_score(y_test, probabilities[:,1])
    else:
        ll  = np.nan
        rc = np.nan
    early_detection_score = general_early_detection_score(X_test, y_test, y_pred)
    df = pd.DataFrame(rpt).T[['precision','recall','f1-score']]
    df.loc['roc_auc', :] = [rc, pd.NA, pd.NA]
    df.loc['log_loss', :] = [ll, pd.NA, pd.NA]
    return y_pred, df, early_detection_score

In [ ]:
data_dict = epyt_flow.data.benchmarks.leakdb.load_data(
    #scenarios_id=["1", "2", "3", "4", "5"],  
    scenarios_id=["1", "2", "3", "4", "5", "6", "7", "8", "9", "10"],  
    #scenarios_id=["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20"],
    use_net1=False,
    return_X_y=True
)

X_list, y_list = [], [] 
for sid, (Xi, yi) in data_dict.items():
    X_list.append(Xi.values if hasattr(Xi, "values") else Xi) #Check for correctness. 
    y_list.append(yi)

X_dummy = np.vstack(X_list)
y_dummy = np.concatenate(y_list)

print(f"Total samples: {X_dummy.shape[0]}, features: {X_dummy.shape[1]}")

X_train_dummy, X_test_dummy, y_train_dummy, y_test_dummy = train_test_split( #Might use datasets given train-test split
    X_dummy, y_dummy,
    test_size=0.20,
    random_state=42,
    stratify=y_dummy
)

clf = mixed_model_classification_fit(100, X_train_dummy, y_train_dummy, n_jobs=8)

In [ ]:
sda = mixed_model_classification_predict(clf, X_test_dummy, y_test_dummy)

In [ ]:
X20, y20 = epyt_flow.data.benchmarks.leakdb.load_data(
    scenarios_id=["30"],
    use_net1=False,
    return_X_y=True
)["30"]

X20 = getattr(X20, "values", X20)

y20_pred = mixed_model_classification_predict(clf, X20, y20, n_jobs=4, print_report=True)

## LeakDB

###  Loading scenarios and placing sensors

In this part a subset of the scenarios from LeakDB is loaded and based on existiong pressure data from the different scenarios, the sensor placements are calculated. The scenarios are split into a train and a testset, based on their ids. The pressure sensors are placed based on the training data, assuming that in a realistic scenario the sensors could only be placed based on a finite set of pressure data.

In [ ]:
num_train_scens = 800

#NOTE: this approach calculates a sensor placement based on multiple scenarios, i.e. all scenarios we look at
#      due to the complexity of the algorithm this will take a lot of time!
scen_train_keys = [str(num) for num in random.sample(range(1, 901), num_train_scens)]
scen_test_keys = list(map(str, range(901, 1001)))
scen_keys = scen_train_keys + scen_test_keys
data = epyt_flow.data.benchmarks.leakdb.load_data(scenarios_id=scen_train_keys, use_net1=False, return_X_y=False)

pressure_column_heads = data[scen_train_keys[0]].columns[:32]
node_pressures = []
for key in scen_train_keys:
    for i in range(len(pressure_column_heads)):
        if key == scen_train_keys[0]:
            node_pressures.append(data[key]['Pressure-Node_' + str(i + 1)])
        else:
            node_pressures[i] = np.concatenate((node_pressures[i], data[key]['Pressure-Node_' + str(i + 1)]))
node_pressures = np.array(node_pressures)
node_pressures.shape

In [ ]:
sensor_placements = calc_sensor_placement_parallel(node_pressures, num_sensors=15, n_jobs=8)
print("Sensor placements:", sensor_placements)

In [12]:
# we can use the results from the bigger list of sensor placements to create a smaller list
sensor_placements_5 = sensor_placements[:5]

In [13]:
sensor_placements_10 = sensor_placements[:10]

In [14]:
configs = load_leakdb_scenarios(scenarios_id=scen_keys, use_net1=False, verbose=True)
scenarios = {}
for i, key in enumerate(scen_keys):
    scenarios[key] = ScenarioSimulator(scenario_config=configs[i])

The topology of the water distribution network does not change between the different scenarios. Here the general topology is inspected.

In [ ]:
scenarios[scen_train_keys[0]].plot_topology()

Run the complete set of simulations:

In [ ]:
scada_data = {}
for key, scenario in scenarios.items():
    scada_data[key] = scenario.run_simulation(verbose=True)

### Machine Learning based Leakage Detection

Prepare the simulation results for calibrating (i.e. creating) a Machine Learning based leakage detection method:

- Create a feature vector (pressure readings at the sensors).
- Create ground-truth labels utilizing the [`time_points_to_one_hot_encoding()`](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.html#epyt_flow.utils.time_points_to_one_hot_encoding) helper function.

The scenarios are already split into a train and test set. Because of that, two feature vectors and lable-sets are created from the start, one of each for the training of the classifier and another pair for the testing.

In [ ]:
X_tmp = [scada_data[key].get_data_pressures()[:, sensor_placements] for key in scen_train_keys]
X_train = np.vstack(X_tmp)
X_test = {key: scada_data[key].get_data_pressures()[:, sensor_placements] for key in scen_test_keys}
y_tmp = []
y_test = {}
for i, key in enumerate(scen_train_keys):
    events_times = [int(t / configs[i].general_params["hydraulic_time_step"])
                    for t in scenarios[key].get_events_active_time_points()]
    y_tmp.append(time_points_to_one_hot_encoding(events_times, total_length=X_tmp[0].shape[0]))
for i, key in enumerate(scen_test_keys):
    events_times = [int(t / configs[i].general_params["hydraulic_time_step"])
                    for t in scenarios[key].get_events_active_time_points()]
    y_test[key] = time_points_to_one_hot_encoding(events_times, total_length=X_test[key].shape[0])
y_train = np.concatenate(y_tmp)
print(f"Total samples: {X_train.shape[0]}, features: {X_train.shape[1]}, samples_y : {y_train.shape[0]}")

In [18]:
X_tmp_5 = [scada_data[key].get_data_pressures()[:, sensor_placements_5] for key in scen_train_keys]
X_train_5 = np.vstack(X_tmp_5)
X_test_5 = {key: scada_data[key].get_data_pressures()[:, sensor_placements_5] for key in scen_test_keys}

In [19]:
X_tmp_10 = [scada_data[key].get_data_pressures()[:, sensor_placements_10] for key in scen_train_keys]
X_train_10 = np.vstack(X_tmp_10)
X_test_10 = {key: scada_data[key].get_data_pressures()[:, sensor_placements_10] for key in scen_test_keys}

The training data is shuffled to avoid biases: 

In [ ]:
X_train, X_train_5, X_train_10, y_train = sklearn.utils.shuffle(
    X_train, X_train_5, X_train_10, y_train,
    random_state=42
)
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

Apply the classifier to the test data (i.e. the test scenarios). Note that one-hot-encoding isn't necessary here, since our classifier is already trained to return one prediction, 0 or 1, per timestep:

### Train classifiers for each different sensor size

In [ ]:
num_trees = 20
mixed_clf = mixed_model_classification_fit(num_trees, X_train, y_train, n_jobs=8)

In [ ]:
mixed_clf_5 = mixed_model_classification_fit(num_trees, X_train_5, y_train, n_jobs=8)

In [ ]:
mixed_clf_10 = mixed_model_classification_fit(num_trees, X_train_10, y_train, n_jobs=8)

In [24]:
general_path =f"plots/LeakDB/{num_train_scens}_scens_{num_trees}_trees/{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}/"
if not os.path.isdir(general_path):
    os.makedirs(general_path)

In [ ]:
# —————————————————————————————————————————————————————
# For each scenario in the test set, evaluate the model and collect metrics.
# metrics are collected in a list of dataframes and then averaged. The result can then be plotted.
# —————————————————————————————————————————————————————
runs = []
counter = 0
alarms_plot_path = general_path + "alarms_with_gt/"
if not os.path.isdir(alarms_plot_path):
    os.makedirs(alarms_plot_path)
ed_scores = []
for sid in scen_test_keys:
    counter = counter + 1

    y_pred, df_metrics, early_detection_score = evaluate_and_collect(mixed_clf, X_test[sid], y_test[sid])

    runs.append(df_metrics)
    ed_scores.append(early_detection_score)

    tn, fp, fn, tp = confusion_matrix(y_test[sid], y_pred, labels=[0,1]).ravel()
    print(f"Scenario {sid}: TN={tn}, FP={fp}, FN={fn}, TP={tp}\n")
    
    plt.figure(figsize=(10, 3))
    plt.plot(y_test[sid], color="red",   label="Ground truth", linewidth=2)
    plt.bar(range(len(y_pred)), y_pred, alpha=0.3, label="Raised alarm")
    plt.legend(loc="upper right")
    plt.ylabel("Leakage indicator")
    plt.yticks([0,1], ["Inactive","Active"])
    plt.xlabel("Time (5 min steps)")
    plt.title(f"Scenario {sid}")
    plt.tight_layout()
    plt.savefig(alarms_plot_path + f"alarms_with_gt_{sid}.png")
    if counter <= 10:
        plt.show() # TODO: save figure instead of showing it
    plt.close() # TODO: is this necessary?
df_all  = pd.concat(runs, keys=range(len(runs)), names=['run','label'])
df_mean = df_all.groupby(level='label').mean()

In [ ]:
# —————————————————————————————————————————————————————
# For the 5 sensors variation. this is a repeat of the above code but with 5 sensors instead of 15
# —————————————————————————————————————————————————————
runs_5 = []
counter = 0
alarms_plot_path = general_path + "alarms_with_gt_5/"
if not os.path.isdir(alarms_plot_path):
    os.makedirs(alarms_plot_path)
ed_scores_5 = []
for sid in scen_test_keys:
    counter = counter + 1

    y_pred_5, df_metrics_5, early_detection_score = evaluate_and_collect(mixed_clf_5, X_test_5[sid], y_test[sid])

    runs_5.append(df_metrics_5)
    ed_scores_5.append(early_detection_score)

    tn, fp, fn, tp = confusion_matrix(y_test[sid], y_pred_5, labels=[0,1]).ravel()
    print(f"Scenario {sid}: TN={tn}, FP={fp}, FN={fn}, TP={tp}\n")
    
    plt.figure(figsize=(10, 3))
    plt.plot(y_test[sid], color="red",   label="Ground truth", linewidth=2)
    plt.bar(range(len(y_pred_5)), y_pred_5, alpha=0.3, label="Raised alarm")
    plt.legend(loc="upper right")
    plt.ylabel("Leakage indicator")
    plt.yticks([0,1], ["Inactive","Active"])
    plt.xlabel("Time (5 min steps)")
    plt.title(f"Scenario {sid}")
    plt.tight_layout()
    plt.savefig(alarms_plot_path + f"alarms_with_gt_{sid}.png")
    if counter <= 10:
        plt.show() # TODO: save figure instead of showing it
    plt.close() # TODO: is this necessary?

df_all_5  = pd.concat(runs_5, keys=range(len(runs_5)), names=['run','label'])
df_mean_5 = df_all_5.groupby(level='label').mean()

In [ ]:
# —————————————————————————————————————————————————————
# For the 10 sensors, this is a repeat of the above code but with 10 sensors instead of 15
# —————————————————————————————————————————————————————
runs_10 = []
ed_scores_10 = []
counter = 0
alarms_plot_path = general_path + "alarms_with_gt_10/"
if not os.path.isdir(alarms_plot_path):
    os.makedirs(alarms_plot_path)
for sid in scen_test_keys:
    counter = counter + 1

    y_pred_10, df_metrics_10, early_detection_score = evaluate_and_collect(mixed_clf_10, X_test_10[sid], y_test[sid])

    runs_10.append(df_metrics_10)
    ed_scores_10.append(early_detection_score)

    tn, fp, fn, tp = confusion_matrix(y_test[sid], y_pred_10, labels=[0,1]).ravel()
    print(f"Scenario {sid}: TN={tn}, FP={fp}, FN={fn}, TP={tp}\n")

    
    plt.figure(figsize=(10, 3))
    plt.plot(y_test[sid], color="red",   label="Ground truth", linewidth=2)
    plt.bar(range(len(y_pred_10)), y_pred_10, alpha=0.3, label="Raised alarm")
    plt.legend(loc="upper right")
    plt.ylabel("Leakage indicator")
    plt.yticks([0,1], ["Inactive","Active"])
    plt.xlabel("Time (5 min steps)")
    plt.title(f"Scenario {sid}")
    plt.tight_layout()
    plt.savefig(alarms_plot_path + f"alarms_with_gt_{sid}.png")
    if counter <= 10:
        plt.show() # TODO: save figure instead of showing it
    plt.close() # TODO: is this necessary?

df_all_10  = pd.concat(runs_10, keys=range(len(runs_10)), names=['run','label'])
df_mean_10 = df_all_10.groupby(level='label').mean()

### Evaluation of the classifier on the LeakDB dataset

In order to evaluate the performance of the leakage detector, we compute a confusion matrix, plot the leaks as well as the alarms raised by the classifier and apply a few other metrics (as seen with the predictions).

Here, we plot the evaluation metrics precision, recall, f1-score, accuracy, roc_auc_score, and the log loss as an average over all test scenarios

In [ ]:
#Disclaimer: Graphs generated using ChatGPT
# —————————————————————————————————————————————————————
# 1) Per‐label metrics (precision, recall, f1‑score)
# —————————————————————————————————————————————————————
label_metrics = ['precision', 'recall', 'f1-score']
sensor_sets = [
    (df_mean_5,  '5 Sensors'),
    (df_mean_10, '10 Sensors'),
    (df_mean,    '15 Sensors'),
]

for m in label_metrics:
    fig, axes = plt.subplots(1, 3, figsize=(18, 4), sharey=True)
    fig.suptitle(f'Avg {m.capitalize()} over {len(runs_5)} Scenarios', fontsize=16)

    for ax, (dfm, label) in zip(axes, sensor_sets):
        df_plot = dfm.drop(index=[r for r in ['roc_auc','log_loss','accuracy'] if r in dfm.index])
        bars = df_plot[m].plot(
            kind='bar', ax=ax,
            edgecolor='black', linewidth=1, legend=False
        )
        ax.set_ylim(0, 1)
        ax.set_title(label, fontsize=14)
        ax.set_xlabel('Class / Summary', fontsize=12)
        if ax is axes[0]:
            ax.set_ylabel(m.capitalize(), fontsize=12)
        ax.grid(axis='y', linestyle='--', alpha=0.5)
        ax.set_xticklabels(df_plot.index, rotation=0, fontsize=10)

        for rect in bars.patches:
            h = rect.get_height()
            ax.text(
                rect.get_x() + rect.get_width()/2,
                h + 0.02,
                f'{h:.2f}',
                ha='center', va='bottom',
                fontsize=10
            )

    plt.tight_layout(rect=[0, 0, 1, 0.93])

    plt.savefig(f"{general_path}{m}.png", dpi=300)
    plt.show()


# —————————————————————————————————————————————————————
# 2) Global metrics (accuracy, roc_auc, log_loss)
# —————————————————————————————————————————————————————
global_metrics = ['accuracy', 'roc_auc', 'log_loss']
sensor_labels  = ['5 Sensors', '10 Sensors', '15 Sensors']

# gather the “precision” column for each global metric row
values = {
    'accuracy': [df_mean_5.loc['accuracy','precision'],
                 df_mean_10.loc['accuracy','precision'],
                 df_mean.loc['accuracy','precision']],
    'roc_auc':   [df_mean_5.loc['roc_auc','precision'],
                  df_mean_10.loc['roc_auc','precision'],
                  df_mean.loc['roc_auc','precision']],
    'log_loss':  [df_mean_5.loc['log_loss','precision'],
                  df_mean_10.loc['log_loss','precision'],
                  df_mean.loc['log_loss','precision']],
}

x = np.arange(len(sensor_labels))

for m in global_metrics:
    fig, ax = plt.subplots(figsize=(6,4))
    ax.bar(x, values[m], width=0.6, edgecolor='black')
    if m in ['accuracy','roc_auc']:
        ax.set_ylim(0, 1)
    ax.set_xticks(x)
    ax.set_xticklabels(sensor_labels)
    ax.set_ylabel(m.replace('_',' ').capitalize())
    ax.set_title(f'Avg {m.replace("_"," ").capitalize()} over {len(runs_5)} Scenarios')
    for i, v in enumerate(values[m]):
        ax.text(i, v + (0.02 if m in ['accuracy','roc_auc'] else 0),
                f'{v:.2f}', ha='center', va='bottom')
    plt.tight_layout()

    plt.savefig(f"{general_path}{m}.png", dpi=300)
    plt.show()

# —————————————————————————————————————————————————————
# 3) Early Detection Score
# —————————————————————————————————————————————————————
for i in range(len(ed_scores)):
    if type(ed_scores[i]) is np.ndarray:
        ed_scores[i] = ed_scores[i][0]
    if type(ed_scores_5[i]) is np.ndarray:
        ed_scores_5[i] = ed_scores_5[i][0]
    if type(ed_scores_10[i]) is np.ndarray:
        ed_scores_10[i] = ed_scores_10[i][0]
mean_ed_scores = [np.mean(ed_scores_5), np.mean(ed_scores_10), np.mean(ed_scores)]
all_scores = [ed_scores_5, ed_scores_10, ed_scores]


fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 5))


for i, scores in enumerate(all_scores):
    axes[i].bar(scen_test_keys[:10], scores[:10])
    axes[i].set_title('Early Detection Score - ' + sensor_labels[i])
    axes[i].set_xlabel('Scenario')
    axes[i].set_ylabel('Score')
    axes[i].set_xticks(range(len(scores[:10])))


plt.tight_layout()

plt.savefig(f"{general_path}early_detection_scores_scens.png", dpi=300)
plt.show()

plt.figure(figsize=(6, 4))
plt.bar(sensor_labels, mean_ed_scores)
plt.title('Mean Scores for Each List')
plt.xlabel('#Sensors')
plt.ylabel('Mean Score')

#scale y-axis
plt.ylim(0, 1.0)


plt.savefig(f"{general_path}early_detection_scores_means.png", dpi=300)
plt.show()


In [ ]:
df_combined = pd.concat(
    [df_mean_5, df_mean_10, df_mean],
    axis=1,
    keys=['5 Sensors', '10 Sensors', '15 Sensors']
)

df_combined.to_csv(f"{general_path}metrics.csv", index=True)
df_ed_scores = pd.DataFrame({'scenarios': scen_test_keys, 'early_detection_score_5': ed_scores_5, 'early_detection_score_10': ed_scores_10, 'early_detection_score': ed_scores})
df_ed_scores.to_csv(f"{general_path}early_detection_scores.csv", index=False)
print(df_combined.to_markdown(floatfmt=".3f"))
print(df_ed_scores.to_markdown(floatfmt=".3f"))

In [30]:
for scenario in scenarios.values():
    scenario.close()

## BattLeDIM

###  Loading scenarios and placing sensors

Here the BattLeDIM data for training and testing is loaded, the sensor placements are calculated and the simulations are run

In [31]:
# use scada date, because data provides the pressure data for 31 nodes, while the simulation for some reason only provides it for 29
data = load_scada_data(False)

In [ ]:
sensor_placements = calc_sensor_placement_parallel(data.get_data_pressures().T, num_sensors=29, n_jobs=8)

In [33]:
sensor_placements_20 = sensor_placements[:20]
sensor_placements_10 = sensor_placements[:10]
sensor_placements_5 = sensor_placements[:5]

In [34]:
num_train_scens = 10
train_configs = []
train_scenarios = []
test_config = load_scenario(True)
test_scenario = ScenarioSimulator(scenario_config=test_config)
for i in range(num_train_scens):
    train_configs.append(load_scenario(False))
    train_scenarios.append(ScenarioSimulator(scenario_config=train_configs[i]))


In [ ]:
# Run the simulations
scada_data_train = [train_scenarios[i].run_simulation(verbose=True) for i in range(10)]
scada_data_test = test_scenario.run_simulation(verbose=True)

### Machine learning based leakage detection

This includes again the preprocessing and fitting of the data from the simulations, as seen for LeakDB.

In [36]:
X_train_BD = np.vstack([scada_data_train[i].get_data_pressures()[:, sensor_placements] for i in range(10)])
X_train_BD_5 = np.vstack([scada_data_train[i].get_data_pressures()[:, sensor_placements_5] for i in range(10)])
X_train_BD_10 = np.vstack([scada_data_train[i].get_data_pressures()[:, sensor_placements_10] for i in range(10)])
X_train_BD_20 = np.vstack([scada_data_train[i].get_data_pressures()[:, sensor_placements_20] for i in range(10)])
X_test_BD = scada_data_test.get_data_pressures()[:, sensor_placements]
X_test_BD_5 = scada_data_test.get_data_pressures()[:, sensor_placements_5]
X_test_BD_10 = scada_data_test.get_data_pressures()[:, sensor_placements_10]
X_test_BD_20 = scada_data_test.get_data_pressures()[:, sensor_placements_20]

y_train_BD = np.concatenate([time_points_to_one_hot_encoding(
    [int(t / train_configs[i].general_params["hydraulic_time_step"]) for t in train_scenarios[i].get_events_active_time_points()],
    total_length=scada_data_train[i].get_data_pressures().shape[0]) for i in range(num_train_scens)])


events_times = [int(t / test_config.general_params["hydraulic_time_step"])
                for t in test_scenario.get_events_active_time_points()]
y_test_BD = time_points_to_one_hot_encoding(events_times, total_length=X_test_BD.shape[0])



In [37]:
# shuffle the data to avoid any bias, as seen in the LeakDB part
X_train_BD, X_train_BD_5, X_train_BD_10, X_train_BD_20, y_train_BD = sklearn.utils.shuffle(
    X_train_BD, X_train_BD_5, X_train_BD_10, X_train_BD_20, y_train_BD,
    random_state=42
)

In [ ]:
# fit the classifier to the data
num_trees = 20
mixed_clf_BD = mixed_model_classification_fit(num_trees, X_train_BD, y_train_BD, n_jobs=8)
mixed_clf_BD_5 = mixed_model_classification_fit(num_trees, X_train_BD_5, y_train_BD, n_jobs=8)
mixed_clf_BD_10 = mixed_model_classification_fit(num_trees, X_train_BD_10, y_train_BD, n_jobs=8)
mixed_clf_BD_20 = mixed_model_classification_fit(num_trees, X_train_BD_20, y_train_BD, n_jobs=8)

In [39]:
train_suspicious_time_points_BD = mixed_model_classification_predict(mixed_clf_BD, X_train_BD, y_train_BD, n_jobs=8)
suspicious_time_points_BD = mixed_model_classification_predict(mixed_clf_BD, X_test_BD, y_test_BD, n_jobs=8)

In [40]:
suspicious_time_points_BD_5 = mixed_model_classification_predict(mixed_clf_BD_5, X_test_BD_5, y_test_BD, n_jobs=8)
suspicious_time_points_BD_10 = mixed_model_classification_predict(mixed_clf_BD_10, X_test_BD_10, y_test_BD, n_jobs=8)
suspicious_time_points_BD_20 = mixed_model_classification_predict(mixed_clf_BD_20, X_test_BD_20, y_test_BD, n_jobs=8)

In [ ]:
runs = []
ed_scores = []

y_pred, df_metrics, early_detection_score = evaluate_and_collect(mixed_clf_BD, X_test_BD, y_test_BD)
runs.append(df_metrics)
ed_scores.append(early_detection_score)

tn, fp, fn, tp = confusion_matrix(y_test_BD, y_pred, labels=[0,1]).ravel()
print(f"test_scenario, all (29) sensors: TN={tn}, FP={fp}, FN={fn}, TP={tp}\n")

df_all  = pd.concat(runs, keys=range(len(runs)), names=['run','label'])
df_mean = df_all.groupby(level='label').mean()

In [ ]:
runs_5 = []
ed_scores_5 = []

y_pred_5, df_metrics_5, early_detection_score = evaluate_and_collect(mixed_clf_BD_5, X_test_BD_5, y_test_BD)

runs_5.append(df_metrics_5)
ed_scores_5.append(early_detection_score)

tn, fp, fn, tp = confusion_matrix(y_test_BD, y_pred_5, labels=[0,1]).ravel()
print(f"test_scenario, 5 sensors: TN={tn}, FP={fp}, FN={fn}, TP={tp}\n")

df_all_5  = pd.concat(runs_5, keys=range(len(runs_5)), names=['run','label'])
df_mean_5 = df_all_5.groupby(level='label').mean()

In [ ]:
runs_10 = []
ed_scores_10 = []

y_pred_10, df_metrics_10, early_detection_score = evaluate_and_collect(mixed_clf_BD_10, X_test_BD_10, y_test_BD)

runs_10.append(df_metrics_10)
ed_scores_10.append(early_detection_score)

tn, fp, fn, tp = confusion_matrix(y_test_BD, y_pred_10, labels=[0,1]).ravel()
print(f"test_scenario, 10 sensors: TN={tn}, FP={fp}, FN={fn}, TP={tp}\n")

df_all_10  = pd.concat(runs_10, keys=range(len(runs_10)), names=['run','label'])
df_mean_10 = df_all_10.groupby(level='label').mean()

In [ ]:
runs_20 = []
ed_scores_20 = []

y_pred_20, df_metrics_20, early_detection_score = evaluate_and_collect(mixed_clf_BD_20, X_test_BD_20, y_test_BD)

runs_20.append(df_metrics_20)
ed_scores_20.append(early_detection_score)

tn, fp, fn, tp = confusion_matrix(y_test_BD, y_pred_20, labels=[0,1]).ravel()
print(f"test_scenario, 20 sensors: TN={tn}, FP={fp}, FN={fn}, TP={tp}\n")

df_all_20  = pd.concat(runs_20, keys=range(len(runs_20)), names=['run','label'])
df_mean_20 = df_all_20.groupby(level='label').mean()

### Evaluation

In [45]:

general_path = f"plots/BattleDIM/{num_train_scens}_scens_{num_trees}_trees/{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}/"
if not os.path.isdir(general_path):
    os.makedirs(general_path)

In [ ]:
#Disclaimer: Graphs generated using ChatGPT
# —————————————————————————————————————————————————————
# 1) Per‐label metrics (precision, recall, f1‑score)
# —————————————————————————————————————————————————————
label_metrics = ['precision', 'recall', 'f1-score']
sensor_sets = [
    (df_mean_5,  '5 Sensors'),
    (df_mean_10, '10 Sensors'),
    (df_mean_20, '20 Sensors'),
    (df_mean,    '29 Sensors'),
]

for m in label_metrics:
    fig, axes = plt.subplots(1, len(sensor_sets), figsize=(18, 4), sharey=True)
    fig.suptitle(f'Avg {m.capitalize()} over {len(runs_5)} Scenario(s)', fontsize=16)

    for ax, (dfm, label) in zip(axes, sensor_sets):
        df_plot = dfm.drop(index=[r for r in ['roc_auc','log_loss','accuracy'] if r in dfm.index])
        bars = df_plot[m].plot(
            kind='bar', ax=ax,
            edgecolor='black', linewidth=1, legend=False
        )
        ax.set_ylim(0, 1)
        ax.set_title(label, fontsize=14)
        ax.set_xlabel('Class / Summary', fontsize=12)
        if ax is axes[0]:
            ax.set_ylabel(m.capitalize(), fontsize=12)
        ax.grid(axis='y', linestyle='--', alpha=0.5)
        ax.set_xticklabels(df_plot.index, rotation=0, fontsize=10)

        for rect in bars.patches:
            h = rect.get_height()
            ax.text(
                rect.get_x() + rect.get_width()/2,
                h + 0.02,
                f'{h:.2f}',
                ha='center', va='bottom',
                fontsize=10
            )

    plt.tight_layout(rect=[0, 0, 1, 0.93])

    plt.savefig(f"{general_path}{m}.png", dpi=300)
    plt.show()


# —————————————————————————————————————————————————————
# 2) Global metrics (accuracy, roc_auc, log_loss)
# —————————————————————————————————————————————————————
global_metrics = ['accuracy', 'roc_auc', 'log_loss']
sensor_labels  = ['5 Sensors', '10 Sensors', '20 Sensors', '29 Sensors']

# gather the “precision” column for each global metric row
values = {
    'accuracy': [df_mean_5.loc['accuracy','precision'],
                 df_mean_10.loc['accuracy','precision'],
                 df_mean_20.loc['accuracy','precision'],
                 df_mean.loc['accuracy','precision']],
    'roc_auc':   [df_mean_5.loc['roc_auc','precision'],
                  df_mean_10.loc['roc_auc','precision'],
                  df_mean_20.loc['roc_auc','precision'],
                  df_mean.loc['roc_auc','precision']],
    'log_loss':  [df_mean_5.loc['log_loss','precision'],
                  df_mean_10.loc['log_loss','precision'],
                  df_mean_20.loc['log_loss','precision'],
                  df_mean.loc['log_loss','precision']],
}

x = np.arange(len(sensor_labels))

for m in global_metrics:
    fig, ax = plt.subplots(figsize=(6,4))
    ax.bar(x, values[m], width=0.6, edgecolor='black')
    if m in ['accuracy','roc_auc']:
        ax.set_ylim(0, 1)
    ax.set_xticks(x)
    ax.set_xticklabels(sensor_labels)
    ax.set_ylabel(m.replace('_',' ').capitalize())
    ax.set_title(f'Avg {m.replace("_"," ").capitalize()} over {len(runs_5)} Scenarios')
    for i, v in enumerate(values[m]):
        ax.text(i, v + (0.02 if m in ['accuracy','roc_auc'] else 0),
                f'{v:.2f}', ha='center', va='bottom')
    plt.tight_layout()

    plt.savefig(f"{general_path}{m}.png", dpi=300)
    plt.show()

# —————————————————————————————————————————————————————
# 3) Early Detection Score
# —————————————————————————————————————————————————————

all_scores = [ed_scores_5, ed_scores_10, ed_scores_20, ed_scores]
all_scores = [score for sens_scores in all_scores for score in sens_scores]


plt.figure(figsize=(6, 4))
plt.bar(sensor_labels, all_scores)
plt.title('Score for Each number of Sensors')
plt.xlabel('#Sensors')
plt.ylabel('Mean Score')

#scale y-axis
plt.ylim(0, 1.0)


plt.savefig(f"{general_path}early_detection_scores.png", dpi=300)
plt.show()


In [ ]:
# todo: save the metrics to a csv file and plot and save the remaining three graphs
plt.figure(figsize=(16, 6))
plt.plot(list(range(len(y_test_BD))), y_test_BD, color="red", label="Ground truth")
plt.bar(list(range(len(suspicious_time_points_BD))), suspicious_time_points_BD, label="Raised alarm")
plt.legend()
plt.ylabel("Leakage indicator")
plt.yticks([0, 1], ["Inactive", "Active"])
plt.xlabel("Time (5min steps)")
plt.savefig(f"{general_path}ground_truth_with raised_alarms_29_sens_BD.png", dpi=300)
plt.show()

plt.figure(figsize=(16, 6))
plt.plot(list(range(len(y_test_BD))), y_test_BD, color="red", label="Ground truth")
plt.bar(list(range(len(suspicious_time_points_BD_20))), suspicious_time_points_BD_20, label="Raised alarm")
plt.legend()
plt.ylabel("Leakage indicator")
plt.yticks([0, 1], ["Inactive", "Active"])
plt.xlabel("Time (5min steps)")
plt.savefig(f"{general_path}ground_truth_with raised_alarms_20_sens_BD.png", dpi=300)

plt.figure(figsize=(16, 6))
plt.plot(list(range(len(y_test_BD))), y_test_BD, color="red", label="Ground truth")
plt.bar(list(range(len(suspicious_time_points_BD_10))), suspicious_time_points_BD_10, label="Raised alarm")
plt.legend()
plt.ylabel("Leakage indicator")
plt.yticks([0, 1], ["Inactive", "Active"])
plt.xlabel("Time (5min steps)")
plt.savefig(f"{general_path}ground_truth_with raised_alarms_10_sens_BD.png", dpi=300)

plt.figure(figsize=(16, 6))
plt.plot(list(range(len(y_test_BD))), y_test_BD, color="red", label="Ground truth")
plt.bar(list(range(len(suspicious_time_points_BD_5))), suspicious_time_points_BD_5, label="Raised alarm")
plt.legend()
plt.ylabel("Leakage indicator")
plt.yticks([0, 1], ["Inactive", "Active"])
plt.xlabel("Time (5min steps)")
plt.savefig(f"{general_path}ground_truth_with raised_alarms_5_sens_BD.png", dpi=300)

In [ ]:
df_combined = pd.concat(
    [df_mean_5, df_mean_10, df_mean_20, df_mean],
    axis=1,
    keys=['5 Sensors', '10 Sensors', '20 Sensors', '29 Sensors']
)

df_combined.to_csv(f"{general_path}metrics.csv", index=True)
df_ed_scores = pd.DataFrame({'scenarios': ['test_scenario'], 'early_detection_score_5': ed_scores_5, 'early_detection_score_10': ed_scores_10, 'early_detection_score_20': ed_scores_20, 'early_detection_score': ed_scores})
df_ed_scores.to_csv(f"{general_path}early_detection_scores.csv", index=False)
print(df_combined.to_markdown(floatfmt=".3f"))
print(df_ed_scores.to_markdown(floatfmt=".3f"))

In [49]:
for scen in train_scenarios:
    scen.close()
test_scenario.close()
    